# Lab 1
Authors
- Francisco Roh
- Bryan Calisto


Build a rule-based chatbot. The bot will do the following activities:

When a greeting is recognized, start the conversation
You’ll give 2 options that the user can ask you about the weather or the stock market
You’ll reply and ask if they have a follow
A user can ask up to 2 questions
You’ll dismiss the user with a greeting


### Declaración de ventana de chatbot

In [380]:
import re
import tkinter as tk

# Configuración de la ventana principal de tkinter
root = tk.Tk()
root.title("Chatbot")
root.geometry("500x400")

# Ventana de chat
chat_window = tk.Text(root, wrap=tk.WORD, state=tk.DISABLED, height=20, width=60)
chat_window.pack(padx=10, pady=10)

# Configuración de colores y estilos para los personajes
chat_window.tag_configure("user", foreground="blue")
chat_window.tag_configure("bot", foreground="purple")


### Funciones de mensajería

In [381]:
def print_robot(texto):
    chat_window.config(state=tk.NORMAL)
    chat_window.insert(tk.END, '🤖: ' + texto + '\n', "bot")
    chat_window.config(state=tk.DISABLED)
    chat_window.see(tk.END)

def salute():
    print_robot("👋 Hello human")
    print_robot('You can ask me 2 questions about weather or stock prices. What would you like to know?')

def dont_understand():
    print_robot("I'm sorry, I don't understand that question.")

def ask_weather_details():
    print_robot("Ok. For which city do you want the weather? And do you want it for today, tomorrow, or next week?. Please specify the city and period correctly, separated by a comma.")
    global current_step
    current_step = "weather_details"

def ask_stock_details():
    print_robot("Ok. For which company's stock price do you want. Enter its symbol (for example, AAPL for Apple Inc.)? And do you want the price of today, last week, or last month?. Please specify the company and period correctly, separated by a comma.")
    global current_step
    current_step = "stock_details"

def dismiss():
    print_robot("👋 See ya!")
    print_robot("------------------- CHAT IS CLOSED --------------------\n-----------------------------------------------------------")
    #root.after(1000, root.destroy)  # Espera 1 segundo antes de cerrar

### Funciones de validación y extracción

In [382]:
def is_greeting(user_input):
    greeting_regex = r"\s*(hello|hi|hey|greetings|wassup|wa+s+a+)\s?(friend|dude|bot|robot)?!*"
    return re.match(greeting_regex, user_input, re.IGNORECASE)

def is_goodbye(user_input):
    return re.search(r'\b(bye|goodbye|see ya|adios|cya|later|exit)\b', user_input, re.IGNORECASE)

def is_weather_question(user_input):
    return re.search(r'\b(weather|rain|sun|cloud|temperature|hot|cold|snow|storm|hurricane)\b', user_input, re.IGNORECASE)

def is_stock_question(user_input):
    return re.search(r'\b(stock|stocks|market|price|shares|value|dow|jones|nasdaq|nyse|s&p)\b', user_input, re.IGNORECASE)

 
period_regex = r"(today|tomorrow|next week|last week|last month)"

city_regex = r"([A-Z][a-záéíóúüñ\s]*)"
company_regex = r"([A-Za-z\s]+)"

def extract_city_period(user_input):
    match = re.match(rf"{city_regex},\s*{period_regex}", user_input, re.IGNORECASE)
    if match:
        city = match.group(1).strip()
        period = match.group(2).strip().lower()
        return city, period
    else:
        return None, None

def extract_company_period(user_input):
    match = re.match(rf"{company_regex},\s*{period_regex}", user_input, re.IGNORECASE)
    if match:
        company = match.group(1).strip()
        period = match.group(2).strip().lower()
        return company, period
    else:
        return None, None


### Funciones de respuesta

#### API para obtener información del clima de las ciudades

In [383]:
import requests

def weather_request(ciudad, period):
    api_key = 'aab2914f534666207f7eaaef6c6ef68f'
    base_url = 'http://api.openweathermap.org/data/2.5/forecast'
    current_weather_url = 'http://api.openweathermap.org/data/2.5/weather'
    
    # Parámetros comunes para todas las solicitudes
    params = {
        'q': ciudad,
        'appid': api_key,
        'units': 'metric',  # Utiliza unidades métricas (grados Celsius)
        'lang': 'en'  # Para obtener la descripción del clima en inglés
    }
    
    if period == "today":
        # Obtener el clima actual
        response = requests.get(current_weather_url, params=params)
        weather_data = response.json()
        if response.status_code == 200:
            return f"Today weather in {ciudad.upper()}: {weather_data['main']['temp']}°C, {weather_data['weather'][0]['description']}"
        else:
            return "City not found or error in request."

    elif period in ["tomorrow", "next week"]:
        # Obtener el pronóstico del clima
        response = requests.get(base_url, params=params)
        weather_data = response.json()
        
        if response.status_code == 200:
            if period == "tomorrow":
                # El pronóstico para mañana está aproximadamente a las 24 horas (índice 8)
                forecast = weather_data['list'][8]
                return f"Tomorrow weather in {ciudad.upper()}: {forecast['main']['temp']}°C, {forecast['weather'][0]['description']}"
            
            elif period == "next week":
                climas = f"Weather in {ciudad.upper()} will be:\n"
                # Pronóstico de los próximos 5 días (cada 24 horas, índice 8 por día)
                for day in range(8, 40, 8):
                    forecast = weather_data['list'][day]
                    climas += f"Date: {forecast['dt_txt'].split()[0]}, Temp: {forecast['main']['temp']}°C, Weather: {forecast['weather'][0]['description']}\n"
                return climas
        else:
            return "City not found or error in request."

    else:
        return "Period not valid. Use 'today', 'tomorrow', or 'next week'."

# Ejemplo de uso
print(weather_request('Miami', 'next week'))


Weather in MIAMI will be:
Date: 2024-08-23, Temp: 29.78°C, Weather: moderate rain
Date: 2024-08-24, Temp: 30.14°C, Weather: overcast clouds
Date: 2024-08-25, Temp: 27.03°C, Weather: moderate rain
Date: 2024-08-26, Temp: 29.27°C, Weather: light rain



#### API para obtener información del precio de las acciones

In [384]:
import requests

def stock_price_request(company, period):
    api_key = 'VTZmbnzOJmC6X7VSf51MnGdNpys4K0ok'
    base_url = 'https://api.polygon.io/v2/aggs/ticker'
    
    # Definir la fecha en función del periodo
    from datetime import datetime, timedelta
    
    today = datetime.now().strftime('%Y-%m-%d')
    
    if period == "today":
        start_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
        end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    elif period == "last week":
        start_date = (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')
        end_date = today
    elif period == "last month":
        start_date = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
        end_date = today
    else:
        return "Period not valid. Use 'today', 'last week', or 'last month'."

    # Construir la URL de la solicitud
    url = f'{base_url}/{company}/range/1/day/{start_date}/{end_date}?apiKey={api_key}'

    # Hacer la solicitud a la API de Polygon.io
    response = requests.get(url)
    stock_data = response.json()

    if response.status_code == 200:
        if 'results' in stock_data and len(stock_data['results']) > 0:
            prices = f"Company: {company.upper()}\n"
            for result in stock_data['results']:
                date = datetime.fromtimestamp(result['t'] / 1000).strftime('%Y-%m-%d')
                price = result['c']  # 'c' es el precio de cierre del día
                prices += f"Date: {date}, Close Price: ${price}\n"
            return prices.strip()
        else:
            return "No data found for the given period."
    else:
        return "Company not found or error in request."

# Ejemplo de uso
print(stock_price_request('META', 'last month'))


Company: META
Date: 2024-07-21, Close Price: $487.4
Date: 2024-07-22, Close Price: $488.69
Date: 2024-07-23, Close Price: $461.27
Date: 2024-07-24, Close Price: $453.41
Date: 2024-07-25, Close Price: $465.7
Date: 2024-07-28, Close Price: $465.71
Date: 2024-07-29, Close Price: $463.19
Date: 2024-07-30, Close Price: $474.83
Date: 2024-07-31, Close Price: $497.74
Date: 2024-08-01, Close Price: $488.14
Date: 2024-08-04, Close Price: $475.73
Date: 2024-08-05, Close Price: $494.09
Date: 2024-08-06, Close Price: $488.92
Date: 2024-08-07, Close Price: $509.63
Date: 2024-08-08, Close Price: $517.77
Date: 2024-08-11, Close Price: $515.95
Date: 2024-08-12, Close Price: $528.54
Date: 2024-08-13, Close Price: $526.76
Date: 2024-08-14, Close Price: $537.33
Date: 2024-08-15, Close Price: $527.42
Date: 2024-08-18, Close Price: $529.28
Date: 2024-08-19, Close Price: $526.73


#### Generación de respuestas

In [385]:
def handle_weather_request(city, period):
    print_robot(f"Perfect!. I am fetching this {period} weather...")
    response = weather_request(city, period)
    print_robot(response)

def handle_stock_request(company, period):
    print_robot(f"Perfect!. I am fetching stock price for {company} for {period}...")
    response = stock_price_request(company, period)
    print_robot(response)


### Flujo

In [386]:
def show_questions_left():
    if n_questions_left > 0:
        print_robot(f"You have {n_questions_left} question(s) left.")

def handle_response(user_input):
    global n_questions_left
    global current_step

    if current_step == "awaiting_greeting":
        if not is_greeting(user_input):
            print_robot("Remember to say a greeting like 'hi' to the bot. Otherwise, it won't talk to you.")
        else:
            salute()
            current_step = "awaiting_question"

    elif current_step == "awaiting_question":
        if is_weather_question(user_input):
            ask_weather_details()
        elif is_stock_question(user_input):
            ask_stock_details()
        elif is_goodbye(user_input):
            dismiss()
            # Estado inicial
            n_questions_left = 2
            current_step = "awaiting_greeting"
            print_robot("Remember to say a greeting like 'hi' to the bot. Otherwise, it won't talk to you.")
        else:
            dont_understand()

    elif current_step == "weather_details":
        city, period = extract_city_period(user_input)
        if city and period:
            handle_weather_request(city, period)
            print_robot('I hope that you was waiting this answer!') 
            n_questions_left -= 1
            if n_questions_left == 0:
                dismiss()
                # Estado inicial
                n_questions_left = 2
                current_step = "awaiting_greeting"
                print_robot("Remember to say a greeting like 'hi' to the bot. Otherwise, it won't talk to you.")
                
            else:
                show_questions_left()
                current_step = "awaiting_question"
                print_robot('You can ask me a final question about weather or stock prices. What would you like to know?')
        elif is_goodbye(user_input):
            dismiss()
            # Estado inicial
            n_questions_left = 2
            current_step = "awaiting_greeting"
            print_robot("Remember to say a greeting like 'hi' to the bot. Otherwise, it won't talk to you.")
        else:
            dont_understand()
            print_robot("Please specify the city and period correctly, separated by a comma.")

    elif current_step == "stock_details":
        company, period = extract_company_period(user_input)
        if company and period:
            handle_stock_request(company, period)
            print_robot('I hope that you was waiting this answer!') 
            n_questions_left -= 1
            if n_questions_left == 0:
                dismiss()
                # Estado inicial
                n_questions_left = 2
                current_step = "awaiting_greeting"
                print_robot("Remember to say a greeting like 'hi' to the bot. Otherwise, it won't talk to you.")
                
            else:
                show_questions_left()
                current_step = "awaiting_question"
                print_robot('You can ask me a final question about weather or stock prices. What would you like to know?')
        elif is_goodbye(user_input):
            dismiss()
            # Estado inicial
            n_questions_left = 2
            current_step = "awaiting_greeting"
            print_robot("Remember to say 'hi' to the bot. Otherwise, it won't talk to you.")
        else:
            dont_understand()
            print_robot("Please specify the company and period correctly, separated by a comma.")

    else:
        dont_understand()


### Función de ingreso de datos

In [387]:
def process_input(event=None):
    user_input = user_entry.get()
    chat_window.config(state=tk.NORMAL)
    chat_window.insert(tk.END, "You: " + user_input + '\n', "user")
    chat_window.config(state=tk.DISABLED)
    chat_window.see(tk.END)
    user_entry.delete(0, tk.END)
    handle_response(user_input)
    user_entry.focus()  # Forzar el foco de nuevo al cuadro de entrada
    
# Cuadro de entrada
user_entry = tk.Entry(root, width=60)
user_entry.pack(padx=10, pady=10)
user_entry.bind("<Return>", process_input)

# Asegurar que el foco siempre esté en el cuadro de entrada
user_entry.focus()

# Reenfocar el cuadro de entrada cuando la ventana se activa
root.bind("<FocusIn>", lambda event: user_entry.focus())

'2994266439808<lambda>'

### Ejecución

In [388]:
# Estado inicial
n_questions_left = 2
current_step = "awaiting_greeting"
print_robot("Remember to say a greeting like 'hi' to the bot. Otherwise, it won't talk to you.")

# Ejecutar la ventana principal de tkinter
root.mainloop()